In [24]:
import easyocr
from PIL import Image

# Crie um leitor (reader)
reader = easyocr.Reader(['pt'])  # Especifica o idioma

# Ler o texto da imagem
results = reader.readtext('6.jpg')

# Imprimir os resultados
for (bbox, text, prob) in results:
    print(f'Texto: {text}, Confiança: {prob}')


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


Texto: DlsEOVERs, Confiança: 0.2485222514511449
Texto: sL, Confiança: 0.13545443916121866
Texto: ProgFoD, Confiança: 0.2931641876061566


In [17]:
import cv2
import easyocr

# Carregar a imagem em escala de cinza
image = cv2.imread('5.jpg', cv2.IMREAD_GRAYSCALE)

image_contraste = cv2.equalizeHist(image)

# Aplicar Unsharp Masking
gaussian_blur = cv2.GaussianBlur(image_contraste, (9, 9), 10.0)
sharpened = cv2.addWeighted(image_contraste, 1.5, gaussian_blur, -0.5, 0)

# Salvar a imagem ajustada
cv2.imwrite('imagem_nitida.jpg', sharpened)

# Ler a imagem ajustada
reader = easyocr.Reader(['pt'])
results = reader.readtext(sharpened, text_threshold=0.7, low_text=0.4, link_threshold=0.4)

# Imprimir os resultados
for (bbox, text, prob) in results:
    print(f'Texto: {text}, Confiança: {prob}')


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


Texto: @YF5J92, Confiança: 0.8342566675176543


In [25]:
import cv2
import easyocr
import numpy as np

# Carregar os nomes das classes
with open("classes.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

# Carregar o modelo YOLO pré-treinado e as classes
net = cv2.dnn.readNet("custom-yolov4-detector_final.weights", "custom-yolov4-detector.cfg")
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

# Carregar a imagem
image = cv2.imread('6.jpg')
height, width, channels = image.shape

# Preparar a imagem para YOLO
blob = cv2.dnn.blobFromImage(image, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
net.setInput(blob)
outs = net.forward(output_layers)

# Processar as detecções
class_ids = []
confidences = []
boxes = []

for out in outs:
    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > 0.5:  # Ajuste o limiar de confiança conforme necessário
            center_x = int(detection[0] * width)
            center_y = int(detection[1] * height)
            w = int(detection[2] * width)
            h = int(detection[3] * height)
            x = int(center_x - w / 2)
            y = int(center_y - h / 2)
            boxes.append([x, y, w, h])
            confidences.append(float(confidence))
            class_ids.append(class_id)

# Aplicar Non-Max Suppression
indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

# Cortar a região da placa
for i in range(len(boxes)):
    if i in indices:
        x, y, w, h = boxes[i]
        label = str(classes[class_ids[i]])
        print(label)
        if label == "placa":  # Certifique-se de que a classe está correta
            roi = image[y:y+h, x:x+w]
            # Salvar a região da placa
            cv2.imwrite('placa.jpg', roi)

# Ler a região da placa usando EasyOCR
reader = easyocr.Reader(['pt'])
results = reader.readtext('placa.jpg')

# Imprimir os resultados
for (bbox, text, prob) in results:
    print(f'Texto: {text}, Confiança: {prob}')


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


placa
Texto: lesl, Confiança: 0.0570419542491436
Texto: Pro8Foo, Confiança: 0.4702001546750024


In [1]:
import os
# Obter o caminho absoluto do arquivo de credenciais na pasta atual do projeto
current_directory = os.getcwd()
credential_path = os.path.join(current_directory, "application_default_credentials.json")

# Definir a variável de ambiente GOOGLE_APPLICATION_CREDENTIALS
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = credential_path


In [ ]:
import os
from google.cloud import vision
import re


# Configura o cliente da API
client = vision.ImageAnnotatorClient()

def extract_license_plate(image_path):
    try:
        with open(image_path, 'rb') as image_file:
            content = image_file.read()

        image = vision.Image(content=content)
        response = client.text_detection(image=image)
        texts = response.text_annotations

        if not texts:
            return "Nenhum texto detectado na imagem."

        full_text = texts[0].description
        print(f"Texto detectado: {full_text}")

        plate_pattern = r'[A-Z]{3}[0-9][A-Z0-9][0-9]{2}'
        matches = re.findall(plate_pattern, full_text)

        if matches:
            return matches
        else:
            return "Nenhuma placa encontrada no texto detectado."
    except Exception as e:
        print(f"Erro ao processar a imagem: {e}")
        return None

# Exemplo de uso
image_path = "8.PNG"  # Substitua pelo caminho da sua imagem
plates = extract_license_plate(image_path)
print(f"Placas encontradas: {plates}")


Texto detectado: ◄ Busca.
21:36
HABILITAÇÃO
Atualizado em: 10/12/2024 - 21:30:10
Verifique autenticidade do QR Code com o app Vio
VÁLIDA EM TODO
O TERRITÓRIO NACIONAL
2259590237
REPÚBLICA FEDERATIVA DO BRASIL
MINISTÉRIO DA INFRAESTRUTURA
DEPARTAMENTO NACIONAL DE TRÂNSITO
CARTEIRA NACIONAL DE HABILITAÇÃO
NOME
ISAC SOARES CAMARA JUNIOR
DOC. IDENTIDADE/ORG EMISSOR/UF
2617795 SSP DF
CPF
016.016.291-25
-FILIAÇÃO
ISAC SOARES CAMARA
DATA NASCIMENTO-
19/09/1987
N° REGISTRO
03931237097
VALIDADE
22/06/2031
B
CAT. HAB.-
1 HABILITAÇÃO
16/09/2006
MIRIAN ANDRELINO SOARES CAM
ARA
PERMISSÃO
ACC
D
OF
Histórico de emissões da CNH
Placas encontradas: Nenhuma placa encontrada no texto detectado.


In [ ]:
import base64
import vertexai
from vertexai.generative_models import GenerativeModel, Part, SafetySetting


def generate():
    vertexai.init(project="guardautomation", location="us-central1")
    model = GenerativeModel(
        "gemini-1.5-pro-002",
    )
    responses = model.generate_content(
        [image1, image2, image3, image4, image5, image6, text1],
        generation_config=generation_config,
        safety_settings=safety_settings,
        stream=True,
    )

    for response in responses:
        print(response.text, end="")

text1 = """Crie uma tabela com a placa, modelo e marca do carro, cor,  nome, cpf. Nome, sobrenome , cpf e rg estão nas imagens posteriores a dos carros, a imagem 1 com a 2 e a 3 com a 4 e assim posteriormente. Responda somente a tabela nada além da tabela."""

generation_config = {
    "max_output_tokens": 8192,
    "temperature": 0,
    "top_p": 0.95,
}

safety_settings = [
    SafetySetting(
        category=SafetySetting.HarmCategory.HARM_CATEGORY_HATE_SPEECH,
        threshold=SafetySetting.HarmBlockThreshold.OFF
    ),
    SafetySetting(
        category=SafetySetting.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT,
        threshold=SafetySetting.HarmBlockThreshold.OFF
    ),
    SafetySetting(
        category=SafetySetting.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT,
        threshold=SafetySetting.HarmBlockThreshold.OFF
    ),
    SafetySetting(
        category=SafetySetting.HarmCategory.HARM_CATEGORY_HARASSMENT,
        threshold=SafetySetting.HarmBlockThreshold.OFF
    ),
]

generate()

| Placa    | Modelo     | Marca    | Cor      | Nome                    | CPF              | RG                |
| :------- | :--------- | :------- | :------- | :---------------------- | :--------------- | :---------------- |
| QYF5J92 | Onix       | Chevrolet | Branco   | Isac Soares Camara Junior | 016.016.291-25 | 2677795 SSP DF   |
| PRQ8F00 | Discovery Sport | Land Rover | Azul Escuro | Rafaela Kuhn Valandro    | 016.718.370-29 | 1077638623 SJS/RS |
| IMPERIAL | Mobi       | Fiat     | Branco   | Gilveano Cota          | 020.856.860-36 | 1092477941 SSP RS |

In [10]:
#listando arquivos do diretorio
from pathlib import Path

diretorio = Path("imagens/documentos")

# Listar todos os arquivos
for arquivo in diretorio.iterdir():
    if arquivo.is_file():
        print(arquivo)

imagens\documentos\1.jpg
imagens\documentos\2.jpg
imagens\documentos\3.jpg


In [9]:
#extraindo metadados
import exifread

def get_image_datetime(image_path):
    with open(image_path, 'rb') as f:
        tags = exifread.process_file(f)

        # A data e hora geralmente estão em tags como 'DateTimeOriginal' ou 'DateTimeDigitized'
        datetime_original = tags.get('EXIF DateTimeOriginal')
        datetime_digitized = tags.get('EXIF DateTimeDigitized')

        if datetime_original:
            return datetime_original.values
        elif datetime_digitized:
            return datetime_digitized.values
        else:
            return "Data não encontrada nos metadados EXIF"

# Substitua 'caminho/para/sua/imagem.jpg' pelo caminho da sua imagem
image_path = 'imagens/documentos/1.jpg'
datetime = get_image_datetime(image_path)
print(datetime)

2024:12:09 22:01:55
